# Gallery Example: Erl/D/k Queue

This example demonstrates an Erlang/D/k queueing system:
- **Arrivals**: Erlang distribution
- **Service**: Deterministic (constant service times)
- **Servers**: k servers (default k=2)
- **Capacity**: Infinite
- **Scheduling**: FCFS

This multiserver queue combines Erlang arrivals (less variable than Poisson) with deterministic service times (zero variance).

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_erldk(k=2): Network {
    """Create Erl/D/k queueing model"""
    val model = Network("Erl/D/k")
// Block 1: nodes
    val source = Source(model, "mySource")
    val queue = Queue(model, "myQueue", SchedStrategy.FCFS)
    val sink = Sink(model, "mySink")
// Block 2: classes
    val oclass = OpenClass(model, "myClass")
// Erlang arrival with mean=1 and order=5
    source.setArrival(oclass, Erlang.fit_mean_and_order(1, 5))
// Deterministic service with mean=2/k (so total capacity matches arrival rate)
    queue.setService(oclass, Det(2/k))
    queue.setNumberOfServers(k)
// Block 3: topology
    model.link(Network.serialRouting(source, queue, sink))
    
    return model
// Create the model with k=2 servers
val model = gallery_erldk(k=2)

## About Erl/D/k Queue

For this queue with k=2 servers:
- **Arrival rate**: λ = 1 (Erlang with mean=1)
- **Service time per server**: 2/k = 1.0 (deterministic)
- **Total service capacity**: k × (1/service_time) = k × (k/2) = k²/2
- **Utilization**: ρ = λ / (k × μ) = 1 / (2 × 1) = 0.5

The combination of:
- Low arrival variability (Erlang SCV = 0.2)
- Zero service variability (deterministic)
- Multiple servers

Results in excellent queueing performance with minimal waiting times.

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")
// CTMC Solver
val solver_ctmc = CTMC(model, "cutoff", 15)
val avg_table_ctmc = solver_ctmc.avgTable
println("\nCTMC Solver:")
println(avg_table_ctmc)
// Fluid Solver
val solver_fluid = FLD(model)
val avg_table_fluid = solver_fluid.avgTable
println("\nFluid Solver:")
println(avg_table_fluid)